# Read the Data Base

In [49]:
import pandas as pd

# Load the Excel file
file_path = r'C:\Users\Fady Kaisar\Desktop\New folder (2)\capstone\Capstone G12\ML\Recipe_and_Properties_of_Leathers\Data - Copy - Copy.xlsx'  # Update with the correct file path
data = pd.read_excel(file_path)

# Display the first few rows to understand the dataset
print("Dataset Preview:")
print(data.head())
print("\nDataset Info:")
print(data.info())


Dataset Preview:
   Soaking_pH  Temperature (°C)  Sodium_sulfide
0         7.5              46.0            0.55
1         7.5              37.0            0.45
2         7.5              15.0            0.50
3         7.5              34.0            0.40
4         8.0              38.0            0.15

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Soaking_pH        500 non-null    float64
 1   Temperature (°C)  500 non-null    float64
 2   Sodium_sulfide    500 non-null    float64
dtypes: float64(3)
memory usage: 11.8 KB
None


# Filtering the Data

In [50]:
# Filter the data for valid Temperature (°C) and Sodium_sulfide values
filtered_data = data[(data['Temperature (°C)'] >= 0 ) & ['Temperature (°C)'] <= 100 ] 
filtered_data = filtered_data[filtered_data['Sodium_sulfide'] >= 0]

# Display the filtered data
print("Filtered Data (outliers removed):")
print(filtered_data)


Filtered Data (outliers removed):
     Soaking_pH  Temperature (°C)  Sodium_sulfide
0      7.500000             46.00        0.550000
1      7.500000             37.00        0.450000
2      7.500000             15.00        0.500000
3      7.500000             34.00        0.400000
4      8.000000             38.00        0.150000
..          ...               ...             ...
495    7.810201              7.89        0.382818
496    7.422589             37.11        0.081155
497    7.477894             19.87        0.592345
498    7.600017             40.06        0.229826
499    7.230641             18.05        0.214058

[494 rows x 3 columns]


C:\Users\Fady Kaisar\AppData\Local\Temp\ipykernel_1904\238645349.py:2: FutureWarning: Logical ops (and, or, xor) between Pandas objects and dtype-less sequences (e.g. list, tuple) are deprecated and will raise in a future version. Wrap the object in a Series, Index, or np.array before operating instead.
  filtered_data = data[(data['Temperature (°C)'] >= 0 ) & ['Temperature (°C)'] <= 100 ]


# Describing the Data

In [51]:
data.describe()

,Soaking_pH,Temperature (°C),Sodium_sulfide
count,500.000000,500.000000,500.000000
mean,8.462390,30.797340,0.711000
std,0.821029,13.510474,0.595659
min,7.166003,3.920000,-0.113061
25%,7.648775,17.980000,0.253959
50%,8.500000,35.775000,0.512298
75%,9.243307,40.952500,0.934965
max,9.914977,62.120000,2.301203


# Define the Values for the Model, X values and Y value

In [52]:
from sklearn.model_selection import train_test_split

# Define features and target
X = filtered_data[['Temperature (°C)', 'Soaking_pH']]
y = filtered_data['Sodium_sulfide']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training Data:")
print(X_train.head())


Training Data:
     Temperature (°C)  Soaking_pH
443             21.87    9.015513
56               4.00    9.600000
354             38.90    7.694716
223             43.01    8.595520
365             45.91    9.538446


# Practising the Model

In [53]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


# Add interaction and higher degree terms
poly = PolynomialFeatures(degree=2, interaction_only=True)
X_poly = poly.fit_transform(X)

# Train a new model with the polynomial features
model_poly = RandomForestRegressor(n_estimators=100, random_state=42)
model_poly.fit(X_poly, y)

# Evaluate the model
y_pred_poly = model_poly.predict(poly.transform(X_test))
mse_poly = mean_squared_error(y_test, y_pred_poly)
r2_poly = r2_score(y_test, y_pred_poly)
print(f"Polynomial Model - MSE: {mse_poly}, R²: {r2_poly}")

Polynomial Model - MSE: 0.008072770299782697, R²: 0.980830183824954


# Firebase Integration

In [54]:
import firebase_admin
from firebase_admin import credentials, db
import numpy as np

# Initialize Firebase
if not firebase_admin._apps:
    cred = credentials.Certificate("credentials.json")  # Replace with your Firebase Admin SDK JSON
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://ml-soaking-app-default-rtdb.firebaseio.com/'  # Replace with your Firebase Database URL
    })

# Firebase reference
firebase_ref = db.reference('/SensorData')  # Reference to the SensorData path


def process_and_update_entry(key, data):
    """
    Process a single Firebase entry and update it with a prediction.
    Args:
        key (str): The Firebase key for the entry.
        data (dict): The data for this entry.
    """
    try:
        # Extract Temperature and pH
        temperature = data.get("Temperature")
        ph_value = data.get("pH")
        prediction = data.get("Prediction")  # Check if prediction already exists

        # Skip entries with existing predictions
        if prediction is not None:
            print(f"Skipping entry {key}: Prediction already exists.")
            return

        # Ensure data exists and is valid
        if temperature is None or ph_value is None:
            print(f"Skipping entry {key}: Missing Temperature or pH")
            return

        # Prepare input for ML model
        input_data = np.array([[float(temperature), float(ph_value)]])
        print(f"Processing entry {key} with input: {input_data}")

        # Predict using the ML model
        prediction = model_poly.predict(poly.transform(input_data))[0]  # Replace with your model logic
        print(f"Prediction for {key}: {prediction}")

        # Update Firebase with the prediction
        firebase_ref.child(key).update({"Prediction": prediction})
        print(f"Updated entry {key} with prediction.")

    except Exception as e:
        print(f"Error processing entry {key}: {e}")


def listener(event):
    """
    This function is triggered whenever there is a change in Firebase data.
    It processes the new or updated entry and adds a prediction.
    """
    if event.event_type == 'put' or event.event_type == 'patch':  # Trigger on data change
        print(f"Change detected in Firebase: {event.path}")
        # The event.data will contain the updated data
        data = event.data
        key = event.path.split("/")[-1]  # Get the key of the entry

        # Process and update the entry with prediction
        process_and_update_entry(key, data)


def main():
    # Set up Firebase listener for changes
    print("Setting up Firebase real-time listener...")
    firebase_ref.listen(listener)  # Listen to changes at the /SensorData node


if __name__ == "__main__":
    main()


Setting up Firebase real-time listener...
Change detected in Firebase: /
Error processing entry : 'NoneType' object has no attribute 'get'


Change detected in Firebase: /-OGQzmgVbLS64s_sBfq-
Processing entry -OGQzmgVbLS64s_sBfq- with input: [[20.25     6.01709]]
Prediction for -OGQzmgVbLS64s_sBfq-: 0.3013218845690634


C:\Users\Fady Kaisar\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(


Updated entry -OGQzmgVbLS64s_sBfq- with prediction.
Change detected in Firebase: /-OGQzmgVbLS64s_sBfq-
Skipping entry -OGQzmgVbLS64s_sBfq-: Prediction already exists.
Change detected in Firebase: /-OGQzpL7wFEwMZEDPgOC
Processing entry -OGQzpL7wFEwMZEDPgOC with input: [[20.25     5.93846]]
Prediction for -OGQzpL7wFEwMZEDPgOC: 0.3013218845690634


C:\Users\Fady Kaisar\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(


Updated entry -OGQzpL7wFEwMZEDPgOC with prediction.
Change detected in Firebase: /-OGQzpL7wFEwMZEDPgOC
Skipping entry -OGQzpL7wFEwMZEDPgOC: Prediction already exists.
Change detected in Firebase: /-OGQzsBVAvDgNJfym48Z
Processing entry -OGQzsBVAvDgNJfym48Z with input: [[20.125    5.85983]]
Prediction for -OGQzsBVAvDgNJfym48Z: 0.36181323813322
Updated entry -OGQzsBVAvDgNJfym48Z with prediction.
Change detected in Firebase: /-OGQzsBVAvDgNJfym48Z
Skipping entry -OGQzsBVAvDgNJfym48Z: Prediction already exists.


C:\Users\Fady Kaisar\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(


Change detected in Firebase: /-OGQzutIO1qAH4tRGiSo
Processing entry -OGQzutIO1qAH4tRGiSo with input: [[20.1875   4.87521]]
Prediction for -OGQzutIO1qAH4tRGiSo: 0.27791787558614856


C:\Users\Fady Kaisar\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(


Updated entry -OGQzutIO1qAH4tRGiSo with prediction.
Change detected in Firebase: /-OGQzutIO1qAH4tRGiSo
Skipping entry -OGQzutIO1qAH4tRGiSo: Prediction already exists.
Change detected in Firebase: /-OGQzxa7lky0dpXg6mMw
Processing entry -OGQzxa7lky0dpXg6mMw with input: [[20.125    5.87009]]
Prediction for -OGQzxa7lky0dpXg6mMw: 0.36181323813322
Updated entry -OGQzxa7lky0dpXg6mMw with prediction.
Change detected in Firebase: /-OGQzxa7lky0dpXg6mMw
Skipping entry -OGQzxa7lky0dpXg6mMw: Prediction already exists.
